### Este notebook tem como função criar uma pasta chamada "Faces Dataset" onde os rostos recortados dos vídeos serão guardados

Inicialmente, carregamos as dependências necessárias

In [1]:
import cv2

import pandas as pd

from facenet_pytorch import MTCNN

from PIL import Image
import glob, os

import torch
import torchvision

# Metodologia para criar o Image Dataset:
- Inicialmente, achar bons valores de threshold para a detecção dos rostos, de forma que se garanta uma confiabilidade alta para todos os vídeos. (garantir que estou encontrando o rosto e estou recortando nos limiares aceitáveis).
- Listar todas as pastas dentro do train (Para se saber quais pastas foram acessadas)
- Listar todos os vídeos dentro de cada pasta (Para se saber quais vídeos foram acessados)
- Abrir frame a frame do vídeo, para cada vídeo
- Aplicar a detecção do rosto (com tamanho padrão definido) => Tamanho de entrada dos modelos pré treinados = 224x224 px
- Recortar a região devolvida do rosto
- Mudar o esquema de cores de RGB para YUV e recortar o canal Y
- Salvar em uma pasta correspondente sendo FAKE ou REAL

# Primeiro passo: Arrumar os valores de threshold

In [2]:
# Definimos um device onde os tensores estarão sendo processados
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [3]:
# Cria uma lista de todas as pastas disponíveis para treinamento
folders = next(os.walk('./Kaggle Dataset/'))[1]
folders[:15]

['dfdc_train_part_0',
 'dfdc_train_part_1',
 'dfdc_train_part_10',
 'dfdc_train_part_11',
 'dfdc_train_part_12',
 'dfdc_train_part_13',
 'dfdc_train_part_14',
 'dfdc_train_part_15',
 'dfdc_train_part_16',
 'dfdc_train_part_17',
 'dfdc_train_part_18',
 'dfdc_train_part_19',
 'dfdc_train_part_2',
 'dfdc_train_part_20',
 'dfdc_train_part_21']

Cria-se uma classe para controlar vídeos, uma função para mostrar vídeos e uma função para realizar o loop de recorte de rostos e salvamento nas pastas.

In [4]:
# Cuida de lidar com o acesso aos vídeos e devolver os paths / labels corretamente
class Videos():
    def __init__(self, folder_path):
        # Guarda o folder_path
        self.folder_path = folder_path
        
        # Guarda a lista de todos os arquivos de videos dentro do folder_path
        self.video_files = glob.glob(folder_path + '/*.mp4')
        
        # Lê o arquivo JSON que contém as informações dos deepfakes naquela pasta
        self.metadata = pd.read_json(folder_path + '/metadata.json').transpose() # Essa transposiçao eh feita pois as colunas e as linhas estao trocadas
        
    def getRandomVideo(self):
        video_path = random.choice(self.video_files)
        video_name = os.path.basename(video_path)
        label = self.metadata.loc[video_name].label
        
        return video_path, video_name, label
        
    def getRealVideo(self, video_name):
        real_video_name = self.metadata.loc[video_name].original
        # Verifica se é NaN, pois caso seja o nome original é o próprio video real
        if pd.isna(real_video_name):
            real_video_name = video_name
        real_video_path = folder_path + '/' + real_video_name
        return real_video_path, real_video_name
    
    def getAllVideosPath(self):
        for video_name, columns in self.metadata.iterrows():
            yield self.folder_path + '/' + video_name, video_name, columns[0] # Label

            
# Mostra o vídeo dado um path e uma label correta respectiva a ele
def showVideo(video_path, label):
    cap = cv2.VideoCapture(video_path)

    # Configura a cor a ser colocada na LABEL
    if label == 'REAL':
        color = (0, 255, 0) # Verde
    else:
        color = (0, 0, 255) # Vermelho    

    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            boxes, _ = mtcnn.detect(Image.fromarray(frame))
            if boxes is not None:
                for box in boxes:
                    cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), color=[0, 255, 0], thickness=5)
                    # Escreve a label no vídeo sob a cabeça do indivíduo
                    cv2.putText(img=frame, text=label, org=(box[0], box[1]), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=color, thickness=2)

            frame = cv2.resize(frame, (1280, 720))
            cv2.imshow('frame', frame)
            #cv2.waitKey(10)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            cv2.destroyAllWindows()
            break

    cap.release()
    cv2.destroyAllWindows()
    
# Esta função permite realiza algo similar à showVideo(), porém ela abre um segundo display de vídeo mostrando a região da face recortada
def showCropFace(video_path, padding=0):
    cap = cv2.VideoCapture(video_path)
    face = None
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret:
            #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            boxes, _ = mtcnn.detect(Image.fromarray(frame))
            if boxes is not None:
                for box in boxes: 
                    face = frame[int(box[1] - padding):int(box[3] + padding), int(box[0] - padding):int(box[2] + padding)]
                    face = cv2.cvtColor(face, cv2.COLOR_BGR2YCrCb)
                    
                    cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), color=[0, 255, 0], thickness=5)
            #cv2.imshow('face', face)
            if face is not None:
                face = cv2.resize(face, (212, 212))
                cv2.imshow('face', face[:, :, 0])
            frame = cv2.resize(frame, (1280, 720))
            cv2.imshow('frame', frame)
            #cv2.waitKey(10)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            cv2.destroyAllWindows()
            break

    cap.release()
    cv2.destroyAllWindows()

# Essa é de fato a função que entra no loop e é responsável por recortar o rosto e o salvar 
# na pasta correspondente, para todos os frames do vídeo. Ela é chamada 1 vez por vídeo.
def saveCropFaces(video_path, video_name, label, padding=0, size=224, check_every_frame=5, size_folder='224px'):
    
    # Instancia um VideoCapture do arquivo presente em video_path (no caso, o vídeo)
    cap = cv2.VideoCapture(video_path)
    
    # Cria um path para salvar os rostos recortados do vídeo. O path nesse caso está sendo './Faces Dataset/224px/FAKE ou REAL'
    class_path = './Faces Dataset/' + size_folder + '/' + label
    
    # Inicializa face como None
    face = None
    
    # Inicia a contagem de frames do vídeo
    video_frame = 1
 
    # Entra num loop que percorre o vídeo até ele acabar
    while(cap.isOpened()):
        # Obtém o próximo frame do vídeo e um retorno que indica se a captura do frame foi um sucesso.
        ret, frame = cap.read()
        if ret:
            # Só recorta o rosto se o frame atual for mod check_every_frame, ou seja, ele só salva de 15 em 15 frames (meio segundo)
            if video_frame == 1 or video_frame % check_every_frame == 0:
                # Inicializa boxes
                boxes = None
                # Utiliza o MTCNN para detectar a bounding box do rosto
                boxes, _ = mtcnn.detect(Image.fromarray(frame))
                # Pode ser que boxes seja None se não identificar rostos
                if boxes is not None:
                    # Para cada box, recorta a região do frame que contém o rosto
                    for box in boxes: 
                        face = frame[
                            int(max(box[1] - padding, 0)):int(max(box[3] + padding, 0)), 
                            int(max(box[0] - padding, 0)):int(max(box[2] + padding, 0))
                        ]
                        # Converte essa região recortada para YCrCb
                        face = cv2.cvtColor(face, cv2.COLOR_BGR2YCrCb)
                        # Aplica um resize para o tamanho desejado, nesse caso, 224x224px
                        face = cv2.resize(face, (size, size))
                    if face is not None:
                        # Salva o rosto na pasta correta. Observe que ele salva apenas face[:, :, 0], indicando o primeiro canal Y (luma)
                        cv2.imwrite(os.path.join(class_path + '/{}_{}.jpg'.format(video_name,video_frame)), face[:, :, 0])
            # Adiciona 1 no video_frame após terminar a análise do frame específico
            video_frame += 1
        else:
            break

    # Solta o objeto do VideoCapture
    cap.release()
    # Destrói as janelas atualmente ativas
    cv2.destroyAllWindows()

Definimos inicialmente a coleta de uma pasta aleatória

In [5]:
# Coletamos uma aleatória de folders
import random
random_folder = random.choice(folders) + '/'

folder_path = './Kaggle Dataset/' + random_folder

# Instaciamos uma objeto da classe Videos aplicado ao path obtido
videos = Videos(folder_path)

## Checamos algum vídeo dentro dessa pasta

Para isso, cria-se um generator que devolve, a cada acesso, um video_path, um video_name e uma label

In [6]:
videos_generator = videos.getAllVideosPath()

E em seguida, utilizamos a função de mostrar os vídeos para verificar e ajustar os thresholds

In [12]:
thresholds = [0.68, 0.78, 0.78]
min_face_size = 150
mtcnn = MTCNN(keep_all=False, device=device, thresholds=thresholds, min_face_size=min_face_size)

In [13]:
video_path, video_name, label = next(videos_generator)
# Podemos passar um padding para verificar a quantidade desejada de recorte ao redo do rosto detectado
showCropFace(video_path, padding=20)

### Agora podemos realizar o loop folder a folder

Selecionamos o primeiro folder

In [163]:
folder = folders[0]
folder_path = './Kaggle Dataset/' + folder

Instanciamos a classe Videos para o primeiro folder

In [14]:
videos = Videos(folder_path)

Criamos um videos_generator para todos os vídeos do folder 0

In [15]:
videos_generator = videos.getAllVideosPath()

Por fim, utilizando o mtcnn com os parâmetros anteriores otimizados, rodamos um loop de recorte de rostos em todos os rostos dos vídeos presentes na pasta. Para auxiliar no acompanhamento do processo, um indicador de porcentagem de 1% em 1% do total de vídeos computados é mostrado (esse valor pode ser alterado).

In [168]:
# Keep_all = False significa resgatar apenas 1 rosto
mtcnn = MTCNN(keep_all=False, device=device, thresholds=[0.68, 0.78, 0.78], min_face_size=150)

print("-------------- Início do folder {} --------------".format(folder))
videos_quantity = len(videos.video_files)
percentage = 1
print_every = int(videos_quantity / (100/percentage))

for n_video, video_data in enumerate(videos_generator):
    if n_video % print_every == 0:
        print("{}: {:.2f}%...".format(folder, round(n_video / videos_quantity * 100)))
    #print("Video: {}".format(video_data[1]))
    saveCropFaces(*video_data, padding=10, size=128, check_every_frame=15)

-------------- Início do folder dfdc_train_part_0 --------------
dfdc_train_part_0: 0.00%...
Video: vpmyeepbep.mp4
Video: fzvpbrzssi.mp4
Video: htorvhbcae.mp4
Video: fckxaqjbxk.mp4
Video: sphirandia.mp4
Video: vsmadeuczx.mp4
Video: ohaqlzfnuv.mp4
Video: komngcqveq.mp4
Video: iafvzgpbix.mp4
Video: uaukglhmje.mp4
Video: syxobtuucp.mp4
Video: dtjcyzgdts.mp4
Video: viuioldtnu.mp4
dfdc_train_part_0: 1.00%...
Video: wnaweyzlqh.mp4
Video: ejhhdlzpjg.mp4
Video: sxyrkshzsg.mp4
Video: uvfkppqsjy.mp4
Video: kedahvzait.mp4
Video: idczhqbbqz.mp4
Video: clzkcmoyhb.mp4
Video: vpjfmetsvn.mp4
Video: muksyyltmg.mp4
Video: ztwlbdwyni.mp4
Video: jaurxwocly.mp4
Video: jkxjvuioek.mp4
Video: dhjnjkzuhq.mp4
dfdc_train_part_0: 2.00%...
Video: xcruhaccxc.mp4
Video: rfpjsloxzg.mp4
Video: kmcdjxmnoa.mp4
Video: vvwuqslvnq.mp4
Video: rpxnyljmsf.mp4
Video: txxungvxms.mp4
Video: vtunvalyji.mp4
Video: ihnhxathkq.mp4
Video: lobxktzhbg.mp4
Video: slhinthqcq.mp4
Video: hqzwudvhih.mp4
Video: xpzfhhwkwb.mp4
Video: sglzgdcj

## Agora, podemos finalmente exportar todo o dataset de vídeos em imagens de rostos. (Pode levar algumas horas para concluir).

In [8]:
mtcnn = MTCNN(keep_all=False, device=device, thresholds=[0.68, 0.78, 0.78], min_face_size=150)

for folder in folders:
    folder_path = './Kaggle Dataset/' + folder
    videos = Videos(folder_path)
    videos_generator = videos.getAllVideosPath()
    print("-------------- Início do folder {} --------------".format(folder))
    videos_quantity = len(videos.video_files)
    percentage = 10
    print_every = int(videos_quantity / (100/percentage))

    for n_video, video_data in enumerate(videos_generator):
        if n_video % print_every == 0:
            print("{}: {:.2f}%...".format(folder, round(n_video / videos_quantity * 100)))
        #print("Video: {}".format(video_data[1]))
        saveCropFaces(*video_data, padding=10, size=224, check_every_frame=15, size_folder='224px')

-------------- Início do folder dfdc_train_part_0 --------------
dfdc_train_part_0: 0.00%...
dfdc_train_part_0: 10.00%...
dfdc_train_part_0: 20.00%...
dfdc_train_part_0: 30.00%...
dfdc_train_part_0: 40.00%...
dfdc_train_part_0: 50.00%...
dfdc_train_part_0: 60.00%...
dfdc_train_part_0: 70.00%...
dfdc_train_part_0: 80.00%...
dfdc_train_part_0: 90.00%...
dfdc_train_part_0: 100.00%...
-------------- Início do folder dfdc_train_part_1 --------------
dfdc_train_part_1: 0.00%...
dfdc_train_part_1: 10.00%...
dfdc_train_part_1: 20.00%...
dfdc_train_part_1: 30.00%...
dfdc_train_part_1: 40.00%...
dfdc_train_part_1: 50.00%...
dfdc_train_part_1: 60.00%...
dfdc_train_part_1: 70.00%...
dfdc_train_part_1: 80.00%...
dfdc_train_part_1: 90.00%...
dfdc_train_part_1: 99.00%...
-------------- Início do folder dfdc_train_part_10 --------------
dfdc_train_part_10: 0.00%...
dfdc_train_part_10: 10.00%...
dfdc_train_part_10: 20.00%...
dfdc_train_part_10: 30.00%...
dfdc_train_part_10: 40.00%...
dfdc_train_part_10

KeyboardInterrupt: 

Aqui o processo foi interrompido pois já se tinha mais de 200 mil imagens ao todo, suficiente para realizar alguns testes iniciais.